In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 


In [2]:
df = pd.read_csv('marketDataNasdaqFutures/NQ_continuous_backadjusted_1m_cleaned.csv')
df

,timestamp,open,high,low,close,volume,symbol
0,2020-07-20 00:00:00+00:00,11712.986537,11716.011798,11711.611418,11712.986537,107.0,NQU0
1,2020-07-20 00:01:00+00:00,11713.261560,11713.261560,11708.861180,11710.786347,112.0,NQU0
2,2020-07-20 00:02:00+00:00,11710.236299,11713.811608,11709.961275,11713.811608,44.0,NQU0
3,2020-07-20 00:03:00+00:00,11712.986537,11713.261560,11710.236299,11710.511323,59.0,NQU0
4,2020-07-20 00:04:00+00:00,11710.511323,11710.786347,11709.411228,11710.511323,47.0,NQU0
...,...,...,...,...,...,...,...
1741997,2025-07-17 23:55:00+00:00,23279.500000,23279.750000,23277.500000,23278.000000,66.0,NQU5
1741998,2025-07-17 23:56:00+00:00,23278.250000,23279.250000,23277.000000,23277.750000,62.0,NQU5
1741999,2025-07-17 23:57:00+00:00,23278.000000,23281.250000,23277.500000,23281.250000,83.0,NQU5
1742000,2025-07-17 23:58:00+00:00,23281.500000,23285.750000,23281.000000,23284.000000,115.0,NQU5


In [3]:
import pandas as pd
from datetime import time, datetime

# =============== CONFIG ==================
# Your raw data frame: df with columns: timestamp, open, high, low, close, volume
# If your df is already loaded, start here:
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce") # Coerce converts non-numeric values to NaN
df = df.dropna(subset=["timestamp"]).sort_values("timestamp").set_index("timestamp")

# ---------------- HOLIDAY LIST (ET full-day blocks) 2020–2025 ----------------
# Notes:
# - These are the standard US holidays where CME equity-index futures
#   run a holiday schedule. We conservatively block the entire ET day.
# - If you want precise early-close/late-open windows per year, replace a given
#   date with an exact (start, end) window for that year.
# - Source references: CME trading-hours portal and CME holiday/clearing advisories. 
#   (We block: New Year’s Day (observed), MLK, Presidents, Good Friday, Memorial,
#    Juneteenth (from 2022), Independence Day (observed), Labor, Thanksgiving, Christmas (observed).)

HOLIDAY_DATES_ET = [
    # ----- 2020 -----
    "2020-01-01",  # New Year’s Day
    "2020-01-20",  # MLK Day
    "2020-02-17",  # Presidents Day
    "2020-04-10",  # Good Friday
    "2020-05-25",  # Memorial Day
    "2020-07-03",  # Independence Day (observed)
    "2020-09-07",  # Labor Day
    "2020-11-26",  # Thanksgiving Day
    "2020-11-27",  # Day after Thanksgiving <- ADDED
    "2020-12-24",  # Christmas Eve <- ADDED
    "2020-12-25",  # Christmas Day

    # ----- 2021 -----
    "2021-01-01",  # New Year’s Day
    "2021-01-18",  # MLK Day
    "2021-02-15",  # Presidents Day
    "2021-04-02",  # Good Friday
    "2021-05-31",  # Memorial Day
    "2021-07-05",  # Independence Day (observed)
    "2021-09-06",  # Labor Day
    "2021-11-25",  # Thanksgiving Day
    "2021-11-26",  # Day after Thanksgiving <- ADDED
    "2021-12-24",  # Christmas Day (observed)

    # ----- 2022 -----
    # "2022-01-01" is Saturday, observed Mon Jan 3 (Stock market open, NQ futures normal)
    # Your list correctly omits Jan 1 and Jan 3, 2022.
    "2022-01-17",  # MLK Day
    "2022-02-21",  # Presidents Day
    "2022-04-15",  # Good Friday
    "2022-05-30",  # Memorial Day
    "2022-06-20",  # Juneteenth (observed)
    "2022-07-04",  # Independence Day
    "2022-09-05",  # Labor Day
    "2022-11-24",  # Thanksgiving Day
    "2022-11-25",  # Day after Thanksgiving <- ADDED
    "2022-12-26",  # Christmas Day (observed)

    # ----- 2023 -----
    "2023-01-02",  # New Year’s Day (observed)
    "2023-01-16",  # MLK Day
    "2023-02-20",  # Presidents Day
    "2023-04-07",  # Good Friday
    "2023-05-29",  # Memorial Day
    "2023-06-19",  # Juneteenth
    "2023-07-03",  # Day before Independence Day <- ADDED
    "2023-07-04",  # Independence Day
    "2023-09-04",  # Labor Day
    "2023-11-23",  # Thanksgiving Day
    "2023-11-24",  # Day after Thanksgiving <- ADDED
    "2023-12-25",  # Christmas Day

    # ----- 2024 -----
    "2024-01-01",  # New Year’s Day
    "2024-01-15",  # MLK Day
    "2024-02-19",  # Presidents Day
    "2024-03-29",  # Good Friday
    "2024-05-27",  # Memorial Day
    "2024-06-19",  # Juneteenth
    "2024-07-03",  # Day before Independence Day <- ADDED
    "2024-07-04",  # Independence Day
    "2024-09-02",  # Labor Day
    "2024-11-28",  # Thanksgiving Day
    "2024-11-29",  # Day after Thanksgiving <- ADDED
    "2024-12-24",  # Christmas Eve <- ADDED
    "2024-12-25",  # Christmas Day

    # ----- 2025 -----
    "2025-01-01",  # New Year’s Day
    "2025-01-20",  # MLK Day
    "2025-02-17",  # Presidents Day
    "2025-04-18",  # Good Friday
    "2025-05-26",  # Memorial Day
    "2025-06-19",  # Juneteenth
    "2025-07-03",  # Day before Independence Day <- ADDED
    "2025-07-04",  # Independence Day
    "2025-09-01",  # Labor Day
    "2025-11-27",  # Thanksgiving Day
    "2025-11-28",  # Day after Thanksgiving <- ADDED
    "2025-12-24",  # Christmas Eve <- ADDED
    "2025-12-25",  # Christmas Day
]

# Build concrete (start,end) ET windows for each holiday date (full day)
HOLIDAY_WINDOWS_ET = [
    (
        pd.Timestamp(d + " 00:00", tz="America/New_York"),
        pd.Timestamp(d + " 23:59:59.999999", tz="America/New_York"),
    )
    for d in HOLIDAY_DATES_ET
]

# =============== FILTER LOGIC ==================
# Convert index to ET for session logic
et_index = df.index.tz_convert("America/New_York")
et_time = et_index.time
et_wday = et_index.weekday  # Monday=0 ... Sunday=6

# Daily Globex maintenance break (NOT tradable): 17:00–18:00 ET
not_maint = ~((et_time >= time(17, 0)) & (et_time < time(18, 0)))

# Weekly structure:
# Mon..Thu: tradable except maintenance
mon_thu = (et_wday >= 0) & (et_wday <= 3)

# Friday: tradable only before 17:00 ET
fri = (et_wday == 4) & (et_time < time(17, 0))

# Sunday: tradable only from 18:00 ET onward
sun = (et_wday == 6) & (et_time >= time(18, 0))

# Saturday: closed (no hours)
# (implicitly excluded; nothing to include on Saturday)
weekly_open = mon_thu | fri | sun

standard_tradable = weekly_open & not_maint

# Holidays overlay (drop anything inside these windows)
holiday_mask = pd.Series(True, index=df.index)
if HOLIDAY_WINDOWS_ET:
    keep = pd.Series(True, index=df.index)
    et_ts = et_index
    for start_et, end_et in HOLIDAY_WINDOWS_ET:
        keep &= ~((et_ts >= start_et) & (et_ts <= end_et)) # Negate truth values in keep that meet the holiday condition
    holiday_mask = keep

# Final filtered DataFrame (UTC index preserved)
df_filtered = df[standard_tradable & holiday_mask].copy()

# Optional: if you’d like an ET-indexed version for inspection:
# df_filtered_et = df_filtered.copy()
# df_filtered_et.index = df_filtered_et.index.tz_convert("America/New_York")
df_filtered.ffill(inplace=True)
df_filtered


,open,high,low,close,volume,symbol
timestamp,,,,,,
2020-07-20 00:00:00+00:00,11712.986537,11716.011798,11711.611418,11712.986537,107.0,NQU0
2020-07-20 00:01:00+00:00,11713.261560,11713.261560,11708.861180,11710.786347,112.0,NQU0
2020-07-20 00:02:00+00:00,11710.236299,11713.811608,11709.961275,11713.811608,44.0,NQU0
2020-07-20 00:03:00+00:00,11712.986537,11713.261560,11710.236299,11710.511323,59.0,NQU0
2020-07-20 00:04:00+00:00,11710.511323,11710.786347,11709.411228,11710.511323,47.0,NQU0
...,...,...,...,...,...,...
2025-07-17 23:55:00+00:00,23279.500000,23279.750000,23277.500000,23278.000000,66.0,NQU5
2025-07-17 23:56:00+00:00,23278.250000,23279.250000,23277.000000,23277.750000,62.0,NQU5
2025-07-17 23:57:00+00:00,23278.000000,23281.250000,23277.500000,23281.250000,83.0,NQU5


In [ ]:
# Build only the z-scored OHLC columns, with past-only info
df_norm = pd.DataFrame(index=df_filtered.index)
for col in ["open", "high", "low", "close"]:
    s  = df_filtered[col].astype(float)
    m  = s.rolling(100, min_periods=1).mean()
    sd = s.rolling(100, min_periods=1).std()
    z  = (s - m) / sd.replace(0, np.nan)
    df_norm[f"{col}_z100"] = z.replace([np.inf, -np.inf], 0.0).fillna(0.0)

df_norm_zscore = df_norm  # same columns as before, no future backfill

# 📊 Feature Documentation for Market ML Model

This document explains all engineered features generated from the OHLCV time-series.  
All features are computed in **UTC** and are **lagged to avoid look-ahead bias**.

---

## ✅ Core Principles

- **All inputs remain in UTC**  
- **No leakage:** Any rolling statistic uses `.shift(1)` to ensure only past data is used  
- **Symbols supported:** Features can be computed for one or multiple tickers  
- **Returns computed on log prices**  
- **Three return variants included:** raw, rolling-z, and volatility-scaled  

---

## 🧠 Feature Categories

The features fall into several groups:

1. **Returns & Return Normalisations**
2. **Volatility & Range Measures**
3. **Momentum / Trend**
4. **Volume & Flow**
5. **VWAP & Fair-Value Distance**
6. **Microstructure Proxies (from OHLCV)**
7. **Seasonality & Time Encoding**
8. **GARCH Model-Based Volatility**

---

## 1. 🔁 Returns & Return Normalisations

| Feature | Description | Notes |
|---------|---------------|--------|
| `ret_1` | 1-bar log return | Lagged by 1 bar |
| `ret_1_z` | Rolling z-score of 1-bar return | Uses past mean & std to normalise |
| `ret_1_volsc` | 1-bar return scaled by √(rolling variance) | Makes returns stationary across regimes |
| `ret_k` | k-bar cumulative log return | `'k'` in config; lagged |
| `ret_k_z` | Rolling z-score of k-bar return | Normalised cumulative return |
| `ret_k_volsc` | k-bar return scaled by rolling variance | Volatility-adjusted |

<font color="green" >These help the model learn return magnitude, normalised behaviour, and regime-adjusted movement. </font>

---

## 2. 📈 Volatility & Range Measures

| Feature | Description |
|---------|--------------|
| `atr` | Average True Range (Wilder’s) – measures price range volatility |
| `parkinson_var` | Parkinson range-based variance using ln(high/low)² |
| `rv` | Realised volatility: √(Σ returns² over window) |
| `vol_of_vol` | Volatility of volatility (stdev of return stdev) |

<font color="green" >Captures short-term, range-based, and realised price volatility and volatility regime shifts.</font>

---

## 3. 📉 Momentum / Trend Indicators

| Feature | Description |
|---------|--------------|
| `ema_fast` | Fast EMA of close (default 12) |
| `ema_slow` | Slow EMA of close (default 26) |
| `macd` | Fast EMA – Slow EMA (trend momentum) |
| `macd_sig` | EMA of MACD (signal line) |
| `macd_div` | MACD – Signal (histogram; trend strength) |
| `ema_spread` | Close minus fast EMA (distance to trend anchor) |

<font color="green">These measure trending behavior, trend acceleration, and trend strength.</font>

---

## 4. 📦 Volume & Flow Indicators

| Feature | Description |
|---------|--------------|
| `vol_z` | Volume z-score (deviation from rolling norm) |
| `obv` | On-Balance Volume – cumulative volume adjusted by direction |
| `cmf` | Chaikin Money Flow – volume-weighted buying/selling pressure |


<font color="green">Captures demand/supply imbalance, momentum supported by volume, and accumulation/distribution phases.</font>

---

## 5. ⚖️ VWAP & Fair-Value Distance

| Feature | Description |
|---------|--------------|
| `vwap_roll` | Rolling VWAP (typical price × volume / sum volume) |
| `dist_vwap` | Close – VWAP (raw deviation) |
| `dist_vwap_z` | Distance to VWAP scaled by ATR (vol-adjusted fair-value gap) |

<font color="green">Captures demand/supply imbalance, momentum supported by volume, and accumulation/distribution phases.</font>

---

## 6. 🧩 Microstructure Proxies (from OHLCV Only)

| Feature | Description |
|---------|--------------|
| `clv` | Close Location Value: (Close–Low)/(High–Low) ∈ [0,1] |
| `gap_pct` | Gap size between prev close and open (absolute % gap) |
| `dir_persist` | Fraction of positive returns in rolling window |

<font color="green">These approximate order-flow or market microstructure signals without needing Level-2 data.</font>

---

## 7. 🕒 Seasonality & Time Encoding (UTC)

| Feature | Description |
|---------|--------------|
| `tod_sin`, `tod_cos` | Time-of-day encoded as cyclical features |
| `dow_sin`, `dow_cos` | Day-of-week encoded cyclically |


<font color="green">Helps the model identify behaviour that depends on time of day or day of week.</font>

---

## 8. 📉 GARCH Model-Based Volatility

| Feature | Description |
|---------|--------------|
| `garch_sigma` | Conditional volatility (σₜ) from best-fit GARCH model |

### How it’s computed:

- A small grid search is run on training data over:  
  `(p,q) ∈ {(1,1), (1,2), (2,1)}` × `{Normal, Student-T}`
- Best model is selected using AIC (or BIC if configured)
- Model is **not refit on test** – parameters are fixed and the full series is filtered
- σₜ is lagged by 1 bar to avoid leakage


<font color="green">Adds a robust statistical volatility estimate that adapts to volatility clustering.</font>

---
## 9. Volume Profile features 
- volprofile library https://pypi.org/project/volprofile/

---
## 10. Technical Analysis features
- Pattern mining using Ta-lib
- double-top, maxima/minima, head & shoulders etc.

---

## 🧼 Cleaning & Safety

- All time-series windows use **`.shift(1)`** to avoid forward-looking leakage
- Infinite and NaN values dropped after feature assembly
- Index remains in **UTC at all times**

---

## 🗂 Returned Output Format

- A DataFrame with all features + a `symbol` column
- A dictionary of best GARCH specs per symbol, e.g.:


## 🧠 Feature reduction
- SHAP for feature importance filtering
- ANOVA F-tests for seeing which models have a significant impact on explaining variance in the response
- Correlation Pruning: remove features >0.9 correlation


In [5]:
df_length = len(df_filtered)
prices = df_filtered['close']

### Returns & Return variants

In [6]:
# Returns & Return Normalisation
# ===============================

# 1 bar regular returns
# ------------------------------
returns = prices.pct_change()
# Remove returns that jump across >2min gap
returns = returns.where(returns.index.to_series().diff() <= pd.Timedelta("2min"), 0.0)
returns = returns.replace([np.inf, -np.inf], 0.0).fillna(0.0)
# Z-score normalisation of regular returns (X-Mew)/SD over multiple windows
# ------------------------------
def zscore(series, window):
    rolling_mean = series.rolling(window, min_periods=1).mean()
    rolling_std = series.rolling(window, min_periods=1).std()
    return (series - rolling_mean) / rolling_std.replace(0, np.nan)

returns_z_20   = zscore(returns, 20)
returns_z_60   = zscore(returns, 60)
returns_z_120  = zscore(returns, 120)
returns_z_240  = zscore(returns, 240)
returns_z_360  = zscore(returns, 360)
returns_z_720  = zscore(returns, 720)

# Volatility adjusted returns 
# ------------------------------
def vol_scaled(series, window):
    rolling_std = series.rolling(window, min_periods=1).std()
    return (series / rolling_std.replace(0, np.nan))

returns_vol_20   = vol_scaled(returns, 20)
returns_vol_60   = vol_scaled(returns, 60)
returns_vol_120  = vol_scaled(returns, 120)
returns_vol_240  = vol_scaled(returns, 240)
returns_vol_360  = vol_scaled(returns, 360)
returns_vol_720  = vol_scaled(returns, 720)

# Log returns log(p_t) - log(p_{t-1})
# ------------------------------
log_returns = np.log(prices) - np.log(prices.shift(1))
log_returns = log_returns.replace([np.inf, -np.inf], 0.0).fillna(0.0)
# Remove returns that jump across >2min gap (assuming 1-min bars), prevents return spikes between sessions/trading and non-trading days
log_returns = log_returns.where(log_returns.index.to_series().diff() <= pd.Timedelta("2min"), 0.0)
log_returns_vol_20   = vol_scaled(log_returns, 20)
log_returns_vol_60   = vol_scaled(log_returns, 60)
log_returns_vol_120  = vol_scaled(log_returns, 120)
log_returns_vol_240  = vol_scaled(log_returns, 240) 
log_returns_vol_360  = vol_scaled(log_returns, 360)
log_returns_vol_720  = vol_scaled(log_returns, 720)

#Z-score normalisation of log returns (X-Mew)/SD over multiple windows
# ------------------------------
log_returns_z_20   = zscore(log_returns, 20)
log_returns_z_60   = zscore(log_returns, 60)
log_returns_z_120  = zscore(log_returns, 120)
log_returns_z_240  = zscore(log_returns, 240)
log_returns_z_360  = zscore(log_returns, 360)
log_returns_z_720  = zscore(log_returns, 720)

# k- bar returns
# ------------------------------
def kbar_simple_return(prices, k):
    return (prices.shift(-k) / prices) - 1.0
returns_k_5   = kbar_simple_return(prices, 5)
returns_k_15  = kbar_simple_return(prices, 15)
returns_k_30  = kbar_simple_return(prices, 30)
returns_k_60  = kbar_simple_return(prices, 60)
returns_k_120 = kbar_simple_return(prices, 120)
returns_k_240 = kbar_simple_return(prices, 240)
returns_k_360 = kbar_simple_return(prices, 360)
returns_k_720 = kbar_simple_return(prices, 720)

# k-bar log returns
# ------------------------------
def kbar_log_return(log_returns, k):
    # Gap in time series can cause large return spikes; we handle this below
    # --------------------------------
    # Detect any gap > 2 min within the future horizon (t, t+k]
    gap_now = prices.index.to_series().diff() > pd.Timedelta("2min")
    gap_in_future = gap_now.rolling(window=k).max().shift(-k).fillna(0).astype(bool)

    # Apply to simple and log k-bar labels
    def safe_numeric(s):
        return s.where(~gap_in_future, 0.0).replace([np.inf, -np.inf], 0.0).fillna(0.0)
    return safe_numeric(log_returns.rolling(k).sum().shift(-k))

log_returns_k_5   = kbar_log_return(log_returns, 5)
log_returns_k_15  = kbar_log_return(log_returns, 15)
log_returns_k_30  = kbar_log_return(log_returns, 30)
log_returns_k_60  = kbar_log_return(log_returns, 60)
log_returns_k_120 = kbar_log_return(log_returns, 120)
log_returns_k_240 = kbar_log_return(log_returns, 240)       
log_returns_k_360 = kbar_log_return(log_returns, 360)
log_returns_k_720 = kbar_log_return(log_returns, 720)

# k-bar z_score returns
# ------------------------------
returns_k_z_5   = zscore(returns_k_5, 20)
returns_k_z_15  = zscore(returns_k_15, 60)
returns_k_z_30  = zscore(returns_k_30, 120)
returns_k_z_60  = zscore(returns_k_60, 240)
returns_k_z_120 = zscore(returns_k_120, 360)
returns_k_z_240 = zscore(returns_k_240, 720)
returns_k_z_360 = zscore(returns_k_360, 1440)
returns_k_z_720 = zscore(returns_k_720, 2880)

# k-bar log z_score returns
# ------------------------------
log_returns_k_z_5   = zscore(log_returns_k_5, 20)
log_returns_k_z_15  = zscore(log_returns_k_15, 60)
log_returns_k_z_30  = zscore(log_returns_k_30, 120)
log_returns_k_z_60  = zscore(log_returns_k_60, 240)
log_returns_k_z_120 = zscore(log_returns_k_120, 360)
log_returns_k_z_240 = zscore(log_returns_k_240, 720)
log_returns_k_z_360 = zscore(log_returns_k_360, 1440)
log_returns_k_z_720 = zscore(log_returns_k_720, 2880)

# k-bar volatility scaled log returns
# ------------------------------
log_returns_k_vol_5   = vol_scaled(log_returns_k_5, 20)
log_returns_k_vol_15  = vol_scaled(log_returns_k_15, 60)
log_returns_k_vol_30  = vol_scaled(log_returns_k_30, 120)
log_returns_k_vol_60  = vol_scaled(log_returns_k_60, 240)
log_returns_k_vol_120 = vol_scaled(log_returns_k_120, 360)
log_returns_k_vol_240 = vol_scaled(log_returns_k_240, 720)
log_returns_k_vol_360 = vol_scaled(log_returns_k_360, 1440)
log_returns_k_vol_720 = vol_scaled(log_returns_k_720, 2880)

# k-bar volatility scaled returns
# ------------------------------
returns_k_vol_5   = vol_scaled(returns_k_5, 20)
returns_k_vol_15  = vol_scaled(returns_k_15, 60)
returns_k_vol_30  = vol_scaled(returns_k_30, 120)               
returns_k_vol_60  = vol_scaled(returns_k_60, 240)
returns_k_vol_120 = vol_scaled(returns_k_120, 360)
returns_k_vol_240 = vol_scaled(returns_k_240, 720)
returns_k_vol_360 = vol_scaled(returns_k_360, 1440)
returns_k_vol_720 = vol_scaled(returns_k_720, 2880) 

# ================== BUILD df_returns_variants ==================

df_returns_variants = pd.DataFrame(index=df_filtered.index)

# 1-bar returns
df_returns_variants["ret_1"] = returns
df_returns_variants["ret_1_log"] = log_returns

# 1-bar Z-scores
df_returns_variants["ret_1_z20"] = returns_z_20
df_returns_variants["ret_1_z60"] = returns_z_60
df_returns_variants["ret_1_z120"] = returns_z_120
df_returns_variants["ret_1_z240"] = returns_z_240
df_returns_variants["ret_1_z360"] = returns_z_360
df_returns_variants["ret_1_z720"] = returns_z_720

# 1-bar volatility scaled
df_returns_variants["ret_1_vol20"] = returns_vol_20
df_returns_variants["ret_1_vol60"] = returns_vol_60
df_returns_variants["ret_1_vol120"] = returns_vol_120
df_returns_variants["ret_1_vol240"] = returns_vol_240
df_returns_variants["ret_1_vol360"] = returns_vol_360
df_returns_variants["ret_1_vol720"] = returns_vol_720


# 1-bar log volatility scaled
df_returns_variants["ret_1_log_vol20"] = log_returns_vol_20
df_returns_variants["ret_1_log_vol60"] = log_returns_vol_60
df_returns_variants["ret_1_log_vol120"] = log_returns_vol_120
df_returns_variants["ret_1_log_vol240"] = log_returns_vol_240
df_returns_variants["ret_1_log_vol360"] = log_returns_vol_360
df_returns_variants["ret_1_log_vol720"] = log_returns_vol_720


# 1-bar log Z-scores
df_returns_variants["ret_1_log_z20"] = log_returns_z_20
df_returns_variants["ret_1_log_z60"] = log_returns_z_60
df_returns_variants["ret_1_log_z120"] = log_returns_z_120
df_returns_variants["ret_1_log_z240"] = log_returns_z_240
df_returns_variants["ret_1_log_z360"] = log_returns_z_360
df_returns_variants["ret_1_log_z720"] = log_returns_z_720


# k-bar simple returns
df_returns_variants["ret_k5"] = returns_k_5
df_returns_variants["ret_k15"] = returns_k_15
df_returns_variants["ret_k30"] = returns_k_30
df_returns_variants["ret_k60"] = returns_k_60
df_returns_variants["ret_k120"] = returns_k_120
df_returns_variants["ret_k240"] = returns_k_240
df_returns_variants["ret_k360"] = returns_k_360
df_returns_variants["ret_k720"] = returns_k_720


# k-bar log returns
df_returns_variants["ret_k5_log"] = log_returns_k_5
df_returns_variants["ret_k15_log"] = log_returns_k_15
df_returns_variants["ret_k30_log"] = log_returns_k_30
df_returns_variants["ret_k60_log"] = log_returns_k_60
df_returns_variants["ret_k120_log"] = log_returns_k_120
df_returns_variants["ret_k240_log"] = log_returns_k_240
df_returns_variants["ret_k360_log"] = log_returns_k_360
df_returns_variants["ret_k720_log"] = log_returns_k_720


# k-bar volatility scaled returns
df_returns_variants["ret_k5_vol"] = returns_k_vol_5
df_returns_variants["ret_k15_vol"] = returns_k_vol_15
df_returns_variants["ret_k30_vol"] = returns_k_vol_30
df_returns_variants["ret_k60_vol"] = returns_k_vol_60
df_returns_variants["ret_k120_vol"] = returns_k_vol_120
df_returns_variants["ret_k240_vol"] = returns_k_vol_240
df_returns_variants["ret_k360_vol"] = returns_k_vol_360
df_returns_variants["ret_k720_vol"] = returns_k_vol_720


# k-bar log volatility scaled returns
df_returns_variants["ret_k5_log_vol"] = log_returns_k_vol_5
df_returns_variants["ret_k15_log_vol"] = log_returns_k_vol_15
df_returns_variants["ret_k30_log_vol"] = log_returns_k_vol_30
df_returns_variants["ret_k60_log_vol"] = log_returns_k_vol_60
df_returns_variants["ret_k120_log_vol"] = log_returns_k_vol_120
df_returns_variants["ret_k240_log_vol"] = log_returns_k_vol_240
df_returns_variants["ret_k360_log_vol"] = log_returns_k_vol_360
df_returns_variants["ret_k720_log_vol"] = log_returns_k_vol_720


# k-bar Z-scores
df_returns_variants["ret_k5_z"] = returns_k_z_5
df_returns_variants["ret_k15_z"] = returns_k_z_15
df_returns_variants["ret_k30_z"] = returns_k_z_30
df_returns_variants["ret_k60_z"] = returns_k_z_60
df_returns_variants["ret_k120_z"] = returns_k_z_120
df_returns_variants["ret_k240_z"] = returns_k_z_240
df_returns_variants["ret_k360_z"] = returns_k_z_360
df_returns_variants["ret_k720_z"] = returns_k_z_720


# k-bar log Z-scores
df_returns_variants["ret_k5_log_z"] = log_returns_k_z_5
df_returns_variants["ret_k15_log_z"] = log_returns_k_z_15
df_returns_variants["ret_k30_log_z"] = log_returns_k_z_30
df_returns_variants["ret_k60_log_z"] = log_returns_k_z_60
df_returns_variants["ret_k120_log_z"] = log_returns_k_z_120
df_returns_variants["ret_k240_log_z"] = log_returns_k_z_240
df_returns_variants["ret_k360_log_z"] = log_returns_k_z_360
df_returns_variants["ret_k720_log_z"] = log_returns_k_z_720


# Final sanity check
assert len(df_returns_variants) == df_length
print("df_returns_variants created with shape:", df_returns_variants.shape)
df_returns_variants.replace(np.nan, 0, inplace=True)
NaN_col =df_returns_variants.isna().sum()
NaN_col =df_returns_variants.isna().sum()

# ================END OF CODE==================


df_returns_variants created with shape: (1691325, 74)


### Range and Volatility Measures

In [7]:
import numpy as np
import pandas as pd

# -----------------------------
# Utilities
# -----------------------------
def ensure_reindex(df: pd.DataFrame, index: pd.Index) -> pd.DataFrame:
    """Align a frame/series to a canonical index (no forward/back fill)."""
    return df.reindex(index)

def downcast_floats(df: pd.DataFrame, float_dtype="float32") -> pd.DataFrame:
    """Downcast float columns to save RAM."""
    out = df.copy()
    for c in out.columns:
        if pd.api.types.is_float_dtype(out[c]):
            out[c] = out[c].astype(float_dtype)
    return out

# -----------------------------
# ATR (Average True Range)
# -----------------------------
def add_atr(
    df: pd.DataFrame,
    n: int = 14,
    method: str = "rma",     # "rma" (Wilder) | "sma"
    pct: bool = True,        # add ATR% column (ATR/close)
    gap_safe: bool = True,   # avoid huge TR across big gaps
    gap_timedelta: str = "2min",
    hi_col: str = "high",
    lo_col: str = "low",
    cl_col: str = "close",
    out_prefix: str = "atr",
    include_tr: bool = False # create *_tr once to avoid duplicate col names
) -> pd.DataFrame:
    assert {hi_col, lo_col, cl_col}.issubset(df.columns), "Missing price columns"
    high  = df[hi_col].astype(float)
    low   = df[lo_col].astype(float)
    close = df[cl_col].astype(float)

    prev_close = close.shift(1)
    if gap_safe:
        gap_mask = df.index.to_series().diff() > pd.Timedelta(gap_timedelta)
        prev_close = prev_close.where(~gap_mask, close)

    hl = (high - low).abs()
    hc = (high - prev_close).abs()
    lc = (low  - prev_close).abs()
    tr = pd.concat([hl, hc, lc], axis=1).max(axis=1)

    method = method.lower()
    if method == "rma":
        atr = tr.ewm(alpha=1.0/n, adjust=False).mean(); tag = "rma"
    elif method == "sma":
        atr = tr.rolling(window=n, min_periods=1).mean(); tag = "sma"
    else:
        raise ValueError("method must be 'rma' or 'sma'")

    out = pd.DataFrame(index=df.index)
    if include_tr:
        out[f"{out_prefix}_tr"] = tr
    out[f"{out_prefix}_{n}_{tag}"] = atr
    if pct:
        out[f"{out_prefix}_{n}_{tag}_pct"] = (atr / close)

    return out.replace([np.inf, -np.inf], np.nan).fillna(0.0)

# -----------------------------
# Parkinson volatility
# -----------------------------
def parkinson_variance(high: pd.Series, low: pd.Series, window: int, min_periods: int | None = None) -> pd.Series:
    ln_hl = np.log(high / low)
    factor = 1.0 / (4.0 * np.log(2.0))
    mp = window if min_periods is None else min_periods
    rolling_sum_sq = ln_hl.pow(2).rolling(window=window, min_periods=mp).sum()
    return factor * (rolling_sum_sq / window)

def parkinson_volatility(high: pd.Series, low: pd.Series, window: int, min_periods: int | None = None) -> pd.Series:
    return np.sqrt(parkinson_variance(high, low, window, min_periods=min_periods))

# -----------------------------
# Realised volatility (sqrt(sum r^2))
# -----------------------------
def realised_volatility(returns: pd.Series, window: int, min_periods: int | None = None) -> pd.Series:
    mp = window if min_periods is None else min_periods
    return (returns.pow(2).rolling(window=window, min_periods=mp).sum()).pow(0.5)

# -----------------------------
# Bollinger Bands
# -----------------------------
def calculate_bollinger_bands(price: pd.Series, windows: list[int] = [14, 30, 50]) -> pd.DataFrame:
    features = {}
    for w in windows:
        sma = price.rolling(window=w).mean()
        std = price.rolling(window=w).std()
        upper = sma + 2*std
        lower = sma - 2*std
        signal = np.where(price > upper, 1, np.where(price < lower, -1, 0))
        features[f'sma_{w}'] = sma
        features[f'bb_width_{w}'] = (upper - lower)
        features[f'bb_signal_{w}'] = signal
    out = pd.DataFrame(features, index=price.index)
    return out.replace([np.inf, -np.inf], np.nan).fillna(0.0)

# -----------------------------
# RSI
# -----------------------------
def compute_rsi(series: pd.Series, window: int, method: str = "wilder") -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    if method == "wilder":
        avg_gain = gain.ewm(alpha=1/window, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/window, adjust=False).mean()
    elif method == "sma":
        avg_gain = gain.rolling(window, min_periods=1).mean()
        avg_loss = loss.rolling(window, min_periods=1).mean()
    else:
        raise ValueError("method must be 'wilder' or 'sma'")

    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi.replace([np.inf, -np.inf], np.nan).fillna(50.0)

def compute_multi_rsi(df: pd.DataFrame, price_col: str = "close", windows: list[int] = [7, 14, 21, 50]) -> pd.DataFrame:
    close = df[price_col].astype(float)
    out = {f"rsi_{w}": compute_rsi(close, window=w, method="wilder") for w in windows}
    return pd.DataFrame(out, index=df.index)

# -----------------------------
# Volatility-of-Volatility (VoV) — fast & safe
# -----------------------------
def add_vov_features_fast(
    df: pd.DataFrame,
    vol_cols: list[str] | None = None,
    windows: tuple[int, ...] = (60, 240),
    log_space: bool = True,
    prefix: str = "vov",
    use_min_periods_full: bool = True,
    eps: float = 1e-12,
) -> pd.DataFrame:
    if vol_cols is None:
        vol_cols = [c for c in df.columns if any(k in c.lower() for k in ("atr", "parkinson", "realised", "rv", "vol"))]
    minp = (lambda w: w) if use_min_periods_full else (lambda w: 1)

    new = {}
    for col in vol_cols:
        base = df[col]
        # If duplicate column names exist, df[col] is a DataFrame; take the first occurrence
        if isinstance(base, pd.DataFrame):
            base = base.iloc[:, 0]
        base = pd.to_numeric(base, errors="coerce").astype("float64")

        for w in windows:
            vlin = base.rolling(w, min_periods=minp(w)).std()
            new[f"{prefix}_{col}_{w}"] = vlin.to_numpy()

            if log_space:
                pos = base > 0
                log_base = pd.Series(np.nan, index=base.index, dtype="float64")
                log_base[pos] = np.log(base[pos] + eps)
                vlog = log_base.rolling(w, min_periods=minp(w)).std()
                new[f"{prefix}_{col}_log_{w}"] = vlog.to_numpy()

    block = pd.DataFrame(new, index=df.index)
    block = block.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    block = downcast_floats(block, "float32")
    return pd.concat([df, block], axis=1, copy=False)

# -----------------------------
# Build the full volatility feature block
# -----------------------------
def build_volatility_features(df_filtered: pd.DataFrame) -> pd.DataFrame:
    assert {'open','high','low','close'}.issubset(df_filtered.columns), "df_filtered must have OHLC"
    canon = df_filtered.index
    prices = df_filtered[['open','high','low','close']].astype(float)

    # ATRs (compute *_tr once to avoid duplicate names)
    df_atr_14_rma = add_atr(prices, n=14, method="rma", pct=True, out_prefix="atr", include_tr=True)
    df_atr_14_sma = add_atr(prices, n=14, method="sma", pct=True, out_prefix="atr_sma", include_tr=False)
    df_atr_24_rma = add_atr(prices, n=24, method="rma", pct=True, out_prefix="atr", include_tr=False)
    df_atr_24_sma = add_atr(prices, n=24, method="sma", pct=True, out_prefix="atr_sma", include_tr=False)
    df_atr_50_rma = add_atr(prices, n=50, method="rma", pct=True, out_prefix="atr", include_tr=False)
    df_atr_50_sma = add_atr(prices, n=50, method="sma", pct=True, out_prefix="atr_sma", include_tr=False)

    atr_frames = [ensure_reindex(x, canon) for x in
                  [df_atr_14_rma, df_atr_14_sma, df_atr_24_rma, df_atr_24_sma, df_atr_50_rma, df_atr_50_sma]]
    df_atr = pd.concat(atr_frames, axis=1, join="inner", copy=False).copy()

    # Parkinson volatility
    H, L, C = prices['high'], prices['low'], prices['close']
    par_wins = [5,10,15,30,60,120,240,380,1380]
    df_par = pd.DataFrame(
        {f"parkinson_vol_{w}": parkinson_volatility(H, L, window=w, min_periods=w) for w in par_wins},
        index=canon
    ).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Realised volatility (log-returns)
    returns = np.log(C).diff()
    rv_wins = [5,10,15,30,60,120,240,380,1380]
    df_rv = pd.DataFrame(
        {f"realised_vol_{w}": realised_volatility(returns, window=w, min_periods=w) for w in rv_wins},
        index=canon
    ).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # RSI & Bollinger
    df_rsi  = compute_multi_rsi(df_filtered, price_col="close", windows=[7,14,21,50]).reindex(canon)
    df_boll = calculate_bollinger_bands(C, windows=[14,30,50]).reindex(canon)

    # Assemble core and de-duplicate any accidental name collisions
    core = pd.concat([df_atr, df_par, df_rv, df_rsi, df_boll], axis=1, join="inner", copy=False).copy()
    core = core.loc[:, ~core.columns.duplicated()].copy()

    # Add VoV over volatility-like columns
    vol_cols = [c for c in core.columns if any(k in c.lower() for k in ("atr", "parkinson", "realised", "rv", "vol"))]
    features = add_vov_features_fast(
        core,
        vol_cols=vol_cols,
        windows=(60, 240),
        log_space=True,
        prefix="vov",
        use_min_periods_full=True,
        eps=1e-12,
    )

    # Final clean + downcast
    features = features.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    features = downcast_floats(features, "float32")

    assert len(features) == len(df_filtered), "Length mismatch after assembly"
    return features

# -----------------------------
# USAGE
# -----------------------------
df_volatility_features = build_volatility_features(df_filtered)
print("df_volatility_features created with shape:", df_volatility_features.shape)
assert len(df_volatility_features) == len(df_filtered), "Length mismatch!"

df_volatility_features.isna().sum()


df_volatility_features created with shape: (1691325, 168)


atr_tr                           0
atr_14_rma                       0
atr_14_rma_pct                   0
atr_sma_14_sma                   0
atr_sma_14_sma_pct               0
                                ..
vov_realised_vol_380_log_240     0
vov_realised_vol_1380_60         0
vov_realised_vol_1380_log_60     0
vov_realised_vol_1380_240        0
vov_realised_vol_1380_log_240    0
Length: 168, dtype: int64

### Momentum & Trend features

In [8]:
# Momentum / Trend Features
# ===============================

# Exponential Moving Averages (EMA)
# ------------------------------
def ema(series: pd.Series, span: int) -> pd.Series:
    """
    Compute Exponential Moving Average (EMA) with standard finance conventions.
    Uses adjust=False for true EMA behaviour.
    """
    return series.ewm(span=span, adjust=False).mean()

close = df_filtered["close"].astype(float)

ema_12   = ema(close, 12)
ema_36   = ema(close, 36)
ema_72   = ema(close, 72)
ema_150  = ema(close, 150)
ema_300  = ema(close, 300)
ema_1000 = ema(close, 1000)

df_ema = pd.DataFrame(index=df_filtered.index)
df_ema["ema_12"]   = ema_12
df_ema["ema_36"]   = ema_36
df_ema["ema_72"]   = ema_72
df_ema["ema_150"]  = ema_150
df_ema["ema_300"]  = ema_300
df_ema["ema_1000"] = ema_1000

assert len(df_ema) == df_length
# MACD (Moving Average Convergence Divergence)
# ------------------------------

def macd(series: pd.Series, fast: int, slow: int, signal: int = 9, prefix: str = "macd"):
    """
    Compute MACD line, signal line (EMA of MACD), and divergence (histogram).
    Returns a DataFrame with columns: <prefix>, <prefix>_signal, <prefix>_div.
    """
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False).mean()
    macd_div = macd_line - macd_signal  # histogram / divergence

    out = pd.DataFrame(index=series.index)
    out[f"{prefix}"] = macd_line
    out[f"{prefix}_signal"] = macd_signal
    out[f"{prefix}_div"] = macd_div
    return out

close = df_filtered["close"].astype(float)

# --- Primary MACD using your windows: fast=12, slow=36, signal=9 ---
df_macd_12_36 = macd(close, fast=12, slow=36, signal=9, prefix="macd_12_36")

# MACDs at multiple regimes using list:
pairs = [(12,36), (36,72), (72,150), (150,300), (300,1000)]
dfs = [df_macd_12_36]
for f,s in pairs[1:]:
    dfs.append(macd(close, fast=f, slow=s, signal=9, prefix=f"macd_{f}_{s}"))

# Concatenate all MACD variants if you generated multiple
df_macd = pd.concat(dfs, axis=1)

# Optional: if using these as ML features, lag by 1 to avoid leakage:
df_macd = df_macd.shift(1).fillna(0.0)
assert len(df_macd) == df_length

# EMA Spreads (absolute and percent)
# ----------------------------------
def ema_spread_abs(fast: pd.Series, slow: pd.Series) -> pd.Series:
    return (fast - slow)

def ema_spread_pct(fast: pd.Series, slow: pd.Series) -> pd.Series:
    return (fast / slow - 1.0).replace([np.inf, -np.inf], 0.0)

pairs = [(12,36), (36,72), (72,150), (150,300), (300,1000)]

df_ema_spread = pd.DataFrame(index=df_filtered.index)

for f, s in pairs:
    fast = df_ema[f"ema_{f}"]
    slow = df_ema[f"ema_{s}"]
    df_ema_spread[f"ema_spread_{f}_{s}"]      = ema_spread_abs(fast, slow)
    df_ema_spread[f"ema_spread_pct_{f}_{s}"]  = ema_spread_pct(fast, slow)

# Optional: align for ML like you did for MACD (avoid leakage)
df_ema_spread = df_ema_spread.shift(1).fillna(0.0)

assert len(df_ema_spread) == df_length

df_momentum_features = pd.concat([df_ema, df_macd, df_ema_spread], axis=1)
assert len(df_momentum_features) == df_length
print("df_momentum_features created with shape:", df_momentum_features.shape)
df_momentum_features.isna().sum()


df_momentum_features created with shape: (1691325, 31)


ema_12                     0
ema_36                     0
ema_72                     0
ema_150                    0
ema_300                    0
ema_1000                   0
macd_12_36                 0
macd_12_36_signal          0
macd_12_36_div             0
macd_36_72                 0
macd_36_72_signal          0
macd_36_72_div             0
macd_72_150                0
macd_72_150_signal         0
macd_72_150_div            0
macd_150_300               0
macd_150_300_signal        0
macd_150_300_div           0
macd_300_1000              0
macd_300_1000_signal       0
macd_300_1000_div          0
ema_spread_12_36           0
ema_spread_pct_12_36       0
ema_spread_36_72           0
ema_spread_pct_36_72       0
ema_spread_72_150          0
ema_spread_pct_72_150      0
ema_spread_150_300         0
ema_spread_pct_150_300     0
ema_spread_300_1000        0
ema_spread_pct_300_1000    0
dtype: int64

### Volume & Order Flow Features

In [9]:
# Volume and Order Flow Features
# ===============================

def add_volume_zscores(
    volume: pd.Series,
    windows: list[int],
    log_space: bool = False,
    deseasonalize_intraday: bool = False,
    clip: float | None = None,
    prefix: str = "vol_z",
    eps: float = 1e-9,
) -> pd.DataFrame:
    """
    Compute volume z-scores over multiple rolling windows.

    Parameters
    ----------
    volume : pd.Series
        Per-bar volume (e.g., 1-min). Index should be a DateTimeIndex (UTC is fine).
    windows : list[int]
        Rolling window sizes (in bars), e.g., [5, 20, 60, 120, 240, 390].
    log_space : bool, default False
        If True, compute z-scores in log(volume) space (more stable).
    deseasonalize_intraday : bool, default False
        If True, remove intraday seasonality by minute-of-day before z-scoring.
        (Linear in level space; in log space, removes the average log-volume pattern.)
    clip : float | None, default None
        If set, clip z-scores to [-clip, clip].
    prefix : str, default "vol_z"
        Prefix for output column names.
    eps : float, default 1e-9
        Small constant to avoid log(0).

    Returns
    -------
    pd.DataFrame
        DataFrame with columns like f"{prefix}_{w}" for each window w.
    """
    vol = volume.astype(float).copy()
    assert isinstance(vol.index, pd.DatetimeIndex), "volume index must be a DateTimeIndex"

    # Build base series for z-scoring (with optional de-seasonalization)
    if log_space:
        logv = np.log(vol.clip(lower=eps))
        if deseasonalize_intraday:
            # remove minute-of-day mean in log space: logv - E[logv | minute_of_day]
            mod = logv.groupby(logv.index.time).transform("mean")
            base = (logv - mod)
        else:
            base = logv
    else:
        if deseasonalize_intraday:
            # remove minute-of-day mean in level space: vol - E[vol | minute_of_day]
            mod = vol.groupby(vol.index.time).transform("mean")
            base = (vol - mod)
        else:
            base = vol

    out = pd.DataFrame(index=vol.index)
    for w in windows:
        m = base.rolling(w, min_periods=1).mean()
        s = base.rolling(w, min_periods=1).std()
        z = (base - m) / s.replace(0, np.nan)

        if clip is not None:
            z = z.clip(-clip, clip)

        z = z.replace([np.inf, -np.inf], 0.0).fillna(0.0)

        name = f"{prefix}_{'log_' if log_space else ''}{w}"
        out[name] = z

    return out

vol = df_filtered["volume"]  # your per-bar volume series
windows = [5, 20, 60, 120, 240, 390]

# Plain volume z-scores
vz = add_volume_zscores(vol, windows=windows, log_space=False, deseasonalize_intraday=False)

# Log-volume, de-seasonalized (recommended)
vz_log = add_volume_zscores(vol, windows=windows, log_space=True, deseasonalize_intraday=True, clip=8)

# On-Balance Volume (OBV)
# ------------------------------
def obv(close: pd.Series, volume: pd.Series) -> pd.Series:
    """
    Compute On-Balance Volume (OBV).

    OBV_t = OBV_{t-1} + sign(close_t - close_{t-1}) * volume_t

    +volume added if price up, -volume if price down, unchanged if flat.
    """
    close = close.astype(float)
    volume = volume.astype(float).fillna(0.0)

    # Price direction: +1 up, -1 down, 0 flat
    direction = (close.diff().fillna(0.0)).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    # OBV cumulative
    obv_series = (direction * volume).cumsum()

    return obv_series
close = df_filtered["close"]
vol = df_filtered["volume"]

df_obv = pd.DataFrame(index=df_filtered.index)
df_obv["obv"] = obv(close, vol)

# enhancements for ML
df_obv["obv_norm"] = df_obv["obv"] / df_obv["obv"].abs().rolling(2000, min_periods=1).max()
df_obv["obv_change"] = df_obv["obv"].diff().fillna(0.0)
def zscore(series, window: int):
    r = series.rolling(window, min_periods=1)
    return (series - r.mean()) / r.std().replace(0, np.nan)

df_obv["obv_z_200"] = zscore(df_obv["obv"], 200).fillna(0.0)

# Chaikin Money Flow (CMF)
# ------------------------------

def chaikin_money_flow(
    high: pd.Series,
    low: pd.Series,
    close: pd.Series,
    volume: pd.Series,
    window: int = 20,
) -> pd.Series:
    """
    Chaikin Money Flow (CMF): volume-weighted buying/selling pressure over a window.
    
    CMF_t = sum_{i=t-w+1..t} [ MFM_i * Volume_i ] / sum_{i=t-w+1..t} [ Volume_i ]
    where MFM_i (Money Flow Multiplier) = ((Close - Low) - (High - Close)) / (High - Low)
    
    - Returns a pandas Series aligned to the input index.
    - Safe for bars with High == Low (treats MFM as 0 for that bar).
    """
    high   = high.astype(float)
    low    = low.astype(float)
    close  = close.astype(float)
    volume = volume.astype(float).fillna(0.0)

    # Money Flow Multiplier (handle high==low as 0)
    denom = (high - low)
    mfm = ((close - low) - (high - close))
    mfm = mfm.where(denom != 0, 0.0) / denom.replace(0, np.nan)

    # Money Flow Volume
    mfv = mfm * volume

    # Rolling sums
    sum_mfv = mfv.rolling(window, min_periods=1).sum()
    sum_vol = volume.rolling(window, min_periods=1).sum()

    # CMF
    cmf = (sum_mfv / sum_vol.replace(0, np.nan)).replace([np.inf, -np.inf], 0.0).fillna(0.0)
    cmf.name = f"cmf_{window}"
    return cmf

cmf_20 = chaikin_money_flow(
    df_filtered["high"],
    df_filtered["low"],
    df_filtered["close"],
    df_filtered["volume"],
    window=20
)

# Add to a DF of features
df_volume_flow = pd.DataFrame(index=df_filtered.index)
df_volume_flow["cmf_20"] = cmf_20

# Optional: multiple windows
for w in (10, 20, 60, 120):
    df_volume_flow[f"cmf_{w}"] = chaikin_money_flow(
        df_filtered["high"], df_filtered["low"], df_filtered["close"], df_filtered["volume"], window=w
    )

# Optional: lag for ML to avoid leakage
df_volume_flow = df_volume_flow.shift(1).fillna(0.0)


# Append to your feature DF
df_vol_z = pd.concat([vz, vz_log, df_obv, df_volume_flow], axis=1)
assert len(df_vol_z) == df_length
print("df_vol_z created with shape:", df_vol_z.shape)
df_vol_z.isna().sum()


df_vol_z created with shape: (1691325, 20)


vol_z_5          0
vol_z_20         0
vol_z_60         0
vol_z_120        0
vol_z_240        0
vol_z_390        0
vol_z_log_5      0
vol_z_log_20     0
vol_z_log_60     0
vol_z_log_120    0
vol_z_log_240    0
vol_z_log_390    0
obv              0
obv_norm         1
obv_change       0
obv_z_200        0
cmf_20           0
cmf_10           0
cmf_60           0
cmf_120          0
dtype: int64

# Volume & Order-Flow Features (exact outputs)

Below are the **column names** created by your code and what each represents.  
Final DF: `df_vol_z = concat([vz, vz_log, df_obv, df_volume_flow])`.

---

## 1) Volume z-scores (level space)
- **vol_z_5, vol_z_20, vol_z_60, vol_z_120, vol_z_240, vol_z_390**  
  Z-score of per-bar **volume** over rolling window *w* (level space, no intraday de-seasonalization).

## 2) Volume z-scores (log space, de-seasonalized)
- **vol_z_log_5, vol_z_log_20, vol_z_log_60, vol_z_log_120, vol_z_log_240, vol_z_log_390**  
  Z-score of **log(volume)** after removing **minute-of-day mean** (stabilized, clipped to ±8).

## 3) On-Balance Volume (OBV) family
- **obv** — Cumulative signed volume (adds volume when price up, subtracts when down).  
- **obv_norm** — `obv` normalized by rolling **max(|obv|)** over 2000 bars (range ≈ -1…1).  
- **obv_change** — First difference of `obv` (per-bar OBV flow).  
- **obv_z_200** — Z-score of `obv` over a 200-bar window.

## 4) Chaikin Money Flow (CMF) — **shifted by 1 bar** (leakage-safe)
- **cmf_10, cmf_20, cmf_60, cmf_120**  
  Volume-weighted buying/selling pressure over window *w* (uses MFM × Volume / sum Volume).

---

### Notes
- `vz` and `vz_log` are **not shifted**; `df_volume_flow` (CMF block) is **shifted by 1 bar**.  
- Windows used: `[5, 20, 60, 120, 240, 390]` for z-scores; `(10, 20, 60, 120)` for CMF.  
- All NaNs/±∞ from divisions are cleaned as per code.


### VWAP & Fair-value distance

In [10]:
# VWAP and fair value distance
# -------------------------------

# -------- rolling VWAP (typical-price weighted) --------
def rolling_vwap(
    df: pd.DataFrame,
    window: int,
    use_typical: bool = True,
    gap_safe: bool = True,
    gap_timedelta: str = "2min",
    hi_col: str = "high",
    lo_col: str = "low",
    cl_col: str = "close",
    vol_col: str = "volume",
) -> pd.Series:
    """
    VWAP_t(window) = sum_{t-window+1..t} (TP * Vol) / sum(Vol),
    TP = (H+L+C)/3 if use_typical else Close.
    Gap-safe: sets vol to 0 on bars that open after a big time gap (> gap_timedelta),
              so gaps don't smear VWAP across sessions.
    """
    assert {hi_col, lo_col, cl_col, vol_col}.issubset(df.columns)
    h = df[hi_col].astype(float)
    l = df[lo_col].astype(float)
    c = df[cl_col].astype(float)
    v = df[vol_col].astype(float).replace(np.nan, 0.0)

    if gap_safe:
        gap_mask = df.index.to_series().diff() > pd.Timedelta(gap_timedelta)
        # zero volume on the first bar after a gap (prevents cross-session blending)
        v = v.where(~gap_mask, 0.0)

    tp = ((h + l + c) / 3.0) if use_typical else c
    tpv = (tp * v)

    num = tpv.rolling(window, min_periods=1).sum()
    den = v.rolling(window, min_periods=1).sum()
    vw = (num / den.replace(0, np.nan))

    return vw.replace([np.inf, -np.inf], np.nan).ffill().replace(np.nan, 0.0)


# -------- lightweight ATR (only if you don't already have one) --------
def _atr_wilder(
    df: pd.DataFrame,
    n: int = 14,
    hi_col: str = "high",
    lo_col: str = "low",
    cl_col: str = "close",
    gap_safe: bool = True,
    gap_timedelta: str = "2min",
) -> pd.Series:
    h = df[hi_col].astype(float)
    l = df[lo_col].astype(float)
    c = df[cl_col].astype(float)
    prev_c = c.shift(1)
    if gap_safe:
        gap_mask = df.index.to_series().diff() > pd.Timedelta(gap_timedelta)
        prev_c = prev_c.where(~gap_mask, c)
    tr = pd.concat([(h - l).abs(), (h - prev_c).abs(), (l - prev_c).abs()], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1.0/n, adjust=False).mean()
    return atr.replace([np.inf, -np.inf], 0.0).replace(np.nan, 0.0)


# -------- feature builder: VWAP, dist, dist/ATR --------
def build_vwap_features(
    df: pd.DataFrame,
    windows: list[int] = (5, 10, 15, 30, 60),
    atr_col: str | None = "atr_14_rma",   # pass your existing ATR column name; if None, auto-compute
    atr_n: int = 14,
    shift_by_one: bool = True,            # good practice to avoid leakage
) -> pd.DataFrame:
    """
    Returns columns per window:
      - vwap_w{w}
      - dist_vwap_w{w}      = close - vwap
      - dist_vwap_z_w{w}    = (close - vwap) / ATR
    """
    assert {"high","low","close","volume"}.issubset(df.columns)
    out = pd.DataFrame(index=df.index)
    close = df["close"].astype(float)

    # ATR source
    if atr_col is not None and atr_col in df.columns:
        atr = df[atr_col].astype(float)
    else:
        atr = _atr_wilder(df, n=atr_n)

    # build per-window features
    for w in windows:
        vwap_w = rolling_vwap(df, window=w, use_typical=True)
        dist_raw = (close - vwap_w)
        dist_z = dist_raw / atr.replace(0, np.nan)

        out[f"vwap_w{w}"] = vwap_w
        out[f"dist_vwap_w{w}"] = dist_raw
        out[f"dist_vwap_z_w{w}"] = dist_z.replace([np.inf, -np.inf], 0.0).replace(np.nan, 0.0)

    # hygiene + optional lag
    out = out.replace([np.inf, -np.inf], 0.0).replace(np.nan, 0.0)
    if shift_by_one:
        out = out.shift(1).replace(np.nan, 0.0)

    return out

# If you already created ATR columns with your add_atr function, pass that column name:
# e.g., df_filtered["atr_14_rma"]
df_vwap_features = build_vwap_features(
    df_filtered,
    windows=[5, 10, 15, 30, 60],
    atr_col="atr_14_rma",   # or None to auto-compute
    atr_n=14,
    shift_by_one=True
)

assert len(df_vwap_features) == df_length
print("df_vwap_features created with shape:", df_vwap_features.shape)
df_vwap_features.isna().sum()


df_vwap_features created with shape: (1691325, 15)


vwap_w5            0
dist_vwap_w5       0
dist_vwap_z_w5     0
vwap_w10           0
dist_vwap_w10      0
dist_vwap_z_w10    0
vwap_w15           0
dist_vwap_w15      0
dist_vwap_z_w15    0
vwap_w30           0
dist_vwap_w30      0
dist_vwap_z_w30    0
vwap_w60           0
dist_vwap_w60      0
dist_vwap_z_w60    0
dtype: int64

### Microstructure Proxies

In [11]:
# Microstructure proxies 
# ===============================

# ------------------------------
# 1) Close Location Value [0,1]
# ------------------------------
def clv_01(
    df: pd.DataFrame,
    hi_col: str = "high",
    lo_col: str = "low",
    cl_col: str = "close",
    neutral: float = 0.5,         # value to use when (high==low)
    clip: bool = True,            # clip to [0,1] for safety
    shift_by_one: bool = True     # lag to avoid leakage if using as ML feature
) -> pd.Series:
    h = df[hi_col].astype(float)
    l = df[lo_col].astype(float)
    c = df[cl_col].astype(float)
    rng = (h - l)

    clv = (c - l) / rng.replace(0, np.nan)
    # fallback when range==0
    clv = clv.fillna(neutral)

    if clip:
        clv = clv.clip(lower=0.0, upper=1.0)

    clv = clv.replace([np.inf, -np.inf], 0.0).fillna(neutral)
    if shift_by_one:
        clv = clv.shift(1).fillna(neutral)
    return clv


# ------------------------------
# 2) Gap percent between prev close and open
#    gap_pct = abs(O - C_{t-1}) / abs(C_{t-1})
#    If only_on_gaps=True, set to 0 unless a time gap > gap_timedelta occurs.
# ------------------------------
def gap_pct(
    df: pd.DataFrame,
    op_col: str = "open",
    cl_col: str = "close",
    only_on_gaps: bool = False,
    gap_timedelta: str = "2min",
    shift_by_one: bool = True
) -> pd.Series:
    o = df[op_col].astype(float)
    c = df[cl_col].astype(float)
    prev_c = c.shift(1)

    g = (o - prev_c).abs() / prev_c.abs().replace(0, np.nan)
    g = g.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    if only_on_gaps:
        gap_mask = df.index.to_series().diff() > pd.Timedelta(gap_timedelta)
        g = g.where(gap_mask, 0.0)

    if shift_by_one:
        g = g.shift(1).fillna(0.0)
    return g


# ------------------------------
# 3) Directional persistence
#    Fraction of positive returns in rolling windows.
#    You can pass a precomputed returns series; otherwise uses close-to-close pct_change().
# ------------------------------
def dir_persist(
    df: pd.DataFrame,
    windows: list[int] = (20, 60, 120),
    returns: pd.Series | None = None,
    cl_col: str = "close",
    gap_safe: bool = True,
    gap_timedelta: str = "2min",
    shift_by_one: bool = True
) -> pd.DataFrame:
    if returns is None:
        c = df[cl_col].astype(float)
        r = c.pct_change()
        if gap_safe:
            # zero out returns across big time gaps to avoid cross-session jumps
            is_gap = df.index.to_series().diff() > pd.Timedelta(gap_timedelta)
            r = r.where(~is_gap, 0.0)
        r = r.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    else:
        r = returns.astype(float).replace([np.inf, -np.inf], 0.0).fillna(0.0)

    pos = (r > 0).astype(float)

    out = pd.DataFrame(index=df.index)
    for w in windows:
        out[f"dir_persist_w{w}"] = pos.rolling(window=w, min_periods=1).mean()

    out = out.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    if shift_by_one:
        out = out.shift(1).fillna(0.0)
    return out

df_microstructure = pd.DataFrame(index=df_filtered.index)

df_microstructure["clv_01"] = clv_01(df_filtered)
df_microstructure["gap_pct"] = gap_pct(df_filtered, only_on_gaps=True, gap_timedelta="10min")

dirp = dir_persist(df_filtered, windows=[20, 60, 120], gap_timedelta="2min")
df_microstructure = pd.concat([df_microstructure, dirp], axis=1)

# Sanity
assert len(df_microstructure) == len(df_filtered)
print("df_microstructure created with shape:", df_microstructure.shape)

df_microstructure.isna().sum()

df_microstructure created with shape: (1691325, 5)


clv_01              0
gap_pct             0
dir_persist_w20     0
dir_persist_w60     0
dir_persist_w120    0
dtype: int64

### Volume Profile

In [12]:
# Volume Profile Features
# ===============================
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
# ---------------------------
# Helpers
# ---------------------------
def _bin_edges(
    prices: np.ndarray,
    nBins: int,
    price_min=None,
    price_max=None,
    tick: float | None = 0.25,   # use 0.25 for NQ
):
    """
    Build bin edges. Handles flat/NaN windows by expanding a tiny range
    (or >= 1 tick) around the price so pmax>pmin always holds.
    """
    prices = np.asarray(prices, dtype=float)
    prices = prices[np.isfinite(prices)]
    if prices.size == 0:
        raise ValueError("No finite prices in window.")

    pmin = float(np.min(prices)) if price_min is None else float(price_min)
    pmax = float(np.max(prices)) if price_max is None else float(price_max)

    if not np.isfinite(pmin) or not np.isfinite(pmax):
        raise ValueError("Invalid price range for volume profile (NaNs).")

    if pmax <= pmin:
        # Flat window → create a minimal band around the price
        p = pmin
        width = (tick if (tick and tick > 0) else max(abs(p) * 1e-6, 1e-6))
        pmin = p - 0.5 * width
        pmax = p + 0.5 * width

    eps = (pmax - pmin) * 1e-12
    return np.linspace(pmin, pmax + eps, nBins + 1)


def _build_vp_df(edges: np.ndarray, agg_vol: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame({
        "minPrice": edges[:-1],
        "maxPrice": edges[1:],
        "aggregateVolume": agg_vol.astype(float)
    })


# ---------------------------
# 1) Price/Volume version
# ---------------------------
def getVP(df: pd.DataFrame, nBins: int = 20) -> pd.DataFrame:
    """
    df must contain: ['price', 'volume'].
    Returns DataFrame with ['minPrice','maxPrice','aggregateVolume'] for each bin.
    """
    assert {"price", "volume"}.issubset(df.columns), "getVP expects columns: price, volume"
    price = df["price"].astype(float).to_numpy()
    vol   = df["volume"].astype(float).fillna(0.0).to_numpy()

    edges = _bin_edges(price, nBins)
    # weighted histogram → sum of volumes per price bin
    agg_vol, _ = np.histogram(price, bins=edges, weights=vol)
    return _build_vp_df(edges, agg_vol)


# ---------------------------
# 2) OHLC/Volume version
# ---------------------------
def getVPWithOHLC(df: pd.DataFrame, nBins: int = 20, tick: float = 0.25) -> pd.DataFrame:
    """
    df must contain: ['open','high','low','close','volume'].
    Distributes each bar’s volume across bins proportional to overlap of [low, high].
    """
    need = {"open","high","low","close","volume"}
    assert need.issubset(df.columns), f"getVPWithOHLC expects columns: {need}"

    low   = df["low"].astype(float).to_numpy()
    high  = df["high"].astype(float).to_numpy()
    close = df["close"].astype(float).to_numpy()
    vol   = df["volume"].astype(float).fillna(0.0).to_numpy()

    prices_for_range = np.concatenate([low, high, close])
    finite = np.isfinite(prices_for_range)
    if not finite.any():
        # Return empty profile; caller should treat as "no features"
        return pd.DataFrame(columns=["minPrice","maxPrice","aggregateVolume"])

    edges = _bin_edges(prices_for_range[finite], nBins, tick=tick)
    agg_vol = np.zeros(nBins, dtype=float)

    for lo, hi, cl, v in zip(low, high, close, vol):
        if v <= 0 or not np.isfinite(v) or not (np.isfinite(lo) and np.isfinite(hi) and np.isfinite(cl)):
            continue
        if hi <= lo:
            # Degenerate bar → assign all vol to bin containing close
            i = np.searchsorted(edges, cl, side="right") - 1
            if 0 <= i < nBins:
                agg_vol[i] += v
            continue
        total_range = hi - lo
        i_start = max(0, np.searchsorted(edges, lo, side="right") - 1)
        i_end   = min(nBins - 1, np.searchsorted(edges, hi, side="left"))
        for i in range(i_start, i_end + 1):
            b_lo, b_hi = edges[i], edges[i+1]
            overlap = max(0.0, min(hi, b_hi) - max(lo, b_lo))
            if overlap > 0:
                agg_vol[i] += v * (overlap / total_range)

    return pd.DataFrame({"minPrice": edges[:-1], "maxPrice": edges[1:], "aggregateVolume": agg_vol})



# ---------------------------
# 3) Top-K bins by volume
# ---------------------------
def getKMaxBars(volprofile_result: pd.DataFrame, k: int) -> pd.DataFrame:
    """
    Returns top-k rows by aggregateVolume (descending).
    """
    assert {"minPrice","maxPrice","aggregateVolume"}.issubset(volprofile_result.columns)
    k = max(1, int(k))
    return volprofile_result.sort_values("aggregateVolume", ascending=False).head(k).reset_index(drop=True)


# ---------------------------
# 4) Unusual Increasing/Decreasing bars
# ---------------------------
def getUnusualIncreasingBars(vp: pd.DataFrame, isUpward: bool, z_thresh: float = 2.0) -> pd.DataFrame:
    """
    Identifies price bins where volume is 'unusually' increasing (isUpward=True)
    or decreasing (isUpward=False) versus neighboring bins.

    Method:
      - Sort by bin mid-price ascending
      - Compute first difference of aggregateVolume across bins
      - Z-score the differences; keep bins where sign matches direction and |z| >= z_thresh
    """
    assert {"minPrice","maxPrice","aggregateVolume"}.issubset(vp.columns)
    df = vp.copy()
    df["mid"] = (df["minPrice"] + df["maxPrice"]) * 0.5
    df = df.sort_values("mid").reset_index(drop=True)

    diff = df["aggregateVolume"].diff()
    mu   = diff.rolling(10, min_periods=1).mean()
    sd   = diff.rolling(10, min_periods=1).std()
    z    = (diff - mu) / sd.replace(0, np.nan)
    z    = z.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    if isUpward:
        mask = (diff > 0) & (z >= z_thresh)
    else:
        mask = (diff < 0) & (z <= -z_thresh)

    out = df.loc[mask, ["minPrice","maxPrice","aggregateVolume"]].copy()
    out["zscore_change"] = z[mask].values
    return out.reset_index(drop=True)


# ---------------------------
# 5) Plot (simple horizontal volume profile)
# ---------------------------
def plot(vp: pd.DataFrame, price: pd.Series | np.ndarray) -> None:
    """
    Simple horizontal bar plot of the profile.
    `price` is used to show current price line.
    """
    import matplotlib.pyplot as plt

    df = vp.copy()
    df["mid"] = (df["minPrice"] + df["maxPrice"]) * 0.5
    y = df["mid"].to_numpy()
    x = df["aggregateVolume"].to_numpy()

    fig, ax = plt.subplots()
    ax.barh(y, x, height=(df["maxPrice"] - df["minPrice"]).to_numpy(), align="center")
    if price is not None and len(price) > 0:
        last = float(price.iloc[-1] if isinstance(price, pd.Series) else price[-1])
        ax.axhline(last, linestyle="--")
    ax.set_xlabel("Aggregate Volume")
    ax.set_ylabel("Price")
    ax.set_title("Volume Profile")
    plt.tight_layout()
    plt.show()


# ---------- helpers (coverage, entropy, concentration) ----------
def _value_area(vp: pd.DataFrame, coverage: float) -> tuple[float, float, float]:
    """
    Given VP df with ['minPrice','maxPrice','aggregateVolume'],
    return (VPOC_mid, VA_low, VA_high) for a target coverage (e.g., 0.40 or 0.70).
    """
    vp = vp.copy()
    vp["mid"] = (vp["minPrice"] + vp["maxPrice"]) * 0.5
    total = float(vp["aggregateVolume"].sum())
    if total <= 0:
        return np.nan, np.nan, np.nan

    # VPOC mid
    vpoc_mid = float(vp.loc[vp["aggregateVolume"].idxmax(), "mid"])

    # Greedy fill of largest bins to reach coverage
    vp_sorted = vp.sort_values("aggregateVolume", ascending=False)
    vp_sorted["cum"] = (vp_sorted["aggregateVolume"].cumsum() / total)
    keep = vp_sorted.loc[vp_sorted["cum"] <= coverage]
    if keep.shape[0] < vp_sorted.shape[0]:
        keep = pd.concat([keep, vp_sorted.iloc[[keep.shape[0]]]])

    if keep.empty:
        return vpoc_mid, np.nan, np.nan

    va_low = float(keep["minPrice"].min())
    va_high = float(keep["maxPrice"].max())
    return vpoc_mid, va_low, va_high

def _profile_entropy(vp: pd.DataFrame) -> float:
    vol = vp["aggregateVolume"].to_numpy(dtype=float)
    s = vol.sum()
    if s <= 0: 
        return 0.0
    p = vol / s
    p = p[p > 0]
    return float(-(p * np.log(p)).sum())

def _topk_share(vp: pd.DataFrame, k: int = 3) -> float:
    vol = vp["aggregateVolume"].sort_values(ascending=False).to_numpy(dtype=float)
    s = vol.sum()
    if s <= 0: 
        return 0.0
    return float(vol[:k].sum() / s)

def _unusual_bins_features(vp: pd.DataFrame, last_close: float, z_thresh: float = 2.0) -> dict:
    """
    Summarize 'unusual' volume changes across bins (both increasing and decreasing).
    Returns scalar features for the current window.
    """
    if vp is None or len(vp) == 0 or not np.isfinite(last_close):
        return {k: 0.0 for k in [
            "vp_unusual_inc_count","vp_unusual_inc_maxz","vp_unusual_inc_share","vp_unusual_inc_near_px",
            "vp_unusual_dec_count","vp_unusual_dec_maxz","vp_unusual_dec_share","vp_unusual_dec_near_px"
        ]}

    # sort by mid-price so "neighbor" means adjacent price bins
    tmp = vp.copy()
    tmp["mid"] = (tmp["minPrice"] + tmp["maxPrice"]) * 0.5
    tmp = tmp.sort_values("mid").reset_index(drop=True)

    diff = tmp["aggregateVolume"].diff()
    mu   = diff.rolling(10, min_periods=1).mean()
    sd   = diff.rolling(10, min_periods=1).std()
    z    = ((diff - mu) / sd.replace(0, np.nan)).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    inc_mask = (diff > 0) & (z >= z_thresh)
    dec_mask = (diff < 0) & (z <= -z_thresh)

    total_vol = float(tmp["aggregateVolume"].sum()) if tmp["aggregateVolume"].sum() > 0 else 1.0
    inc_vol = float(tmp.loc[inc_mask, "aggregateVolume"].sum())
    dec_vol = float(tmp.loc[dec_mask, "aggregateVolume"].sum())

    # locate current price's bin
    idx_px = np.searchsorted(tmp["maxPrice"].to_numpy(), last_close)
    idx_px = np.clip(idx_px, 0, len(tmp)-1)  # safe index

    return {
        "vp_unusual_inc_count": float(inc_mask.sum()),
        "vp_unusual_inc_maxz": float(z[inc_mask].max() if inc_mask.any() else 0.0),
        "vp_unusual_inc_share": float(inc_vol / total_vol),
        "vp_unusual_inc_near_px": float(inc_mask.iloc[idx_px]) if len(tmp) else 0.0,

        "vp_unusual_dec_count": float(dec_mask.sum()),
        "vp_unusual_dec_maxz": float((-z[dec_mask]).max() if dec_mask.any() else 0.0),  # magnitude
        "vp_unusual_dec_share": float(dec_vol / total_vol),
        "vp_unusual_dec_near_px": float(dec_mask.iloc[idx_px]) if len(tmp) else 0.0,
    }


# ---------- single-window feature extractor ----------
def _vp_features_for_window(
    df_window: pd.DataFrame,
    n_bins: int,
    coverages: list[float],
    topk: int,
    atr_col: str | None = None  # <- This hint is misleading but left as-is
):
    feats = {}
    try:
        vp = getVPWithOHLC(df_window[["open","high","low","close","volume"]], nBins=n_bins, tick=0.25)
        if vp is None or len(vp) == 0:
            return feats
    except Exception:
        return feats

    ent = _profile_entropy(vp)
    conc_k = _topk_share(vp, k=topk)
    last_close = float(df_window["close"].iloc[-1])
    feats.update(_unusual_bins_features(vp, last_close, z_thresh=2.0))

    # optional ATR (for normalising distances)
    last_atr = None


    # ------------------- FIX: robust ATR lookup -------------------
    if atr_col is not None:
        try:
            # Get the index of the last row of the current window slice
            last_index_for_window = df_window.index[-1]
            
            # Look up that index in the passed ATR Series
            val = float(atr_col.loc[last_index_for_window])
            last_atr = val if np.isfinite(val) and val > 0 else None
        except (KeyError, IndexError, TypeError):
            # KeyError: index not found in atr_col
            # IndexError: df_window was empty
            # TypeError: atr_col was not a Series (just in case)
            last_atr = None
    # ------------------- END OF FIX -------------------

    for cov in coverages:
        vpoc_mid, va_low, va_high = _value_area(vp, cov)

        vpoc_dist = (last_close - vpoc_mid) if np.isfinite(vpoc_mid) else np.nan
        vpoc_dist_pct = (vpoc_dist / last_close) if (np.isfinite(vpoc_dist) and last_close != 0) else np.nan
        va_width = (va_high - va_low) if (np.isfinite(va_high) and np.isfinite(va_low)) else np.nan
        va_width_pct = (va_width / last_close) if (np.isfinite(va_width) and last_close != 0) else np.nan

        # New: inside-VA flag and location code
        in_va = float(np.isfinite(va_low) and np.isfinite(va_high) and (va_low <= last_close <= va_high))
        loc_code = (
            -1.0 if (np.isfinite(va_low) and last_close < va_low) else
             1.0 if (np.isfinite(va_high) and last_close > va_high) else
             0.0 if in_va == 1.0 else np.nan
        )

        # Optional: distance normalised by ATR
        vpoc_dist_atr = (vpoc_dist / last_atr) if (last_atr is not None and np.isfinite(vpoc_dist)) else np.nan

        tag = f"{int(cov*100)}"  # "40" or "70"
        feats.update({
            f"vp_vpoc_{tag}": vpoc_mid,
            f"vp_val_{tag}": va_low,
            f"vp_vah_{tag}": va_high,
            f"vp_va_width_{tag}": va_width,
            f"vp_va_width_pct_{tag}": va_width_pct,
            f"vp_vpoc_dist_{tag}": vpoc_dist,
            f"vp_vpoc_dist_pct_{tag}": vpoc_dist_pct,
            f"vp_in_va_{tag}": in_va,               # <- NEW
            f"vp_loc_{tag}": loc_code,              # <- NEW (-1 below, 0 inside, +1 above)
            f"vp_vpoc_dist_atr_{tag}": vpoc_dist_atr,  # <- NEW (if ATR provided)
            f"vp_above_vah_{tag}": float(last_close > va_high) if np.isfinite(va_high) else 0.0,
            f"vp_below_val_{tag}": float(last_close < va_low) if np.isfinite(va_low) else 0.0,
        })

    feats["vp_entropy"] = ent
    feats["vp_topk_share"] = conc_k
    return feats


# ---------- PARALLEL rolling builder (drop-in replacement) ----------
def build_custom_vp_features(
    df: pd.DataFrame,
    windows: list[int] = (5, 10, 15, 30, 60),
    coverages: list[float] = (0.40, 0.70),
    n_bins: int = 24,
    topk: int = 3,
    stride: int = 1,            # compute every 'stride' bars; forward-fill in between
    shift_by_one: bool = True,  # lag features one bar to avoid leakage
    n_jobs: int = -1,           # NEW: parallel workers (-1 = all cores)
    backend: str = "loky",      # NEW: "loky" (processes) or "threading" (no pickling of df)
    batch_size: str | int = "auto"  # NEW: joblib batching
) -> pd.DataFrame:
    """
    Rolling Volume-Profile features using custom getVPWithOHLC (no binary deps).
    df must have: ['open','high','low','close','volume'] and DateTimeIndex.
    Returns a DataFrame aligned with df.index.
    Parallelized version — identical outputs to the serial implementation.
    """
    assert {"open","high","low","close","volume"}.issubset(df.columns)
    out = pd.DataFrame(index=df.index)

    # tiny wrapper so each task only needs an index
    def _compute_at_index(i: int, w: int):
        # mirrors your exact slicing & call
        sl = df.iloc[i + 1 - w : i + 1]
        # This call is unchanged. 'df_volatility_features' is assumed
        # to be in the scope where this function is defined.
        return _vp_features_for_window(sl, n_bins=n_bins, coverages=list(coverages), topk=topk, atr_col=df_volatility_features['atr_14_rma'])

    for w in windows:
        # pre-allocate rows of dicts to keep positions identical
        rows = [{} for _ in range(len(df))]
        idxs = list(df.index)

        # which indices actually compute features (same condition as before)
        compute_idxs = [i for i in range(len(df)) if (i + 1) >= w and (i % stride) == 0]

        if compute_idxs:
            # run tasks in parallel
            results = Parallel(n_jobs=n_jobs, backend=backend, batch_size=batch_size)(
                delayed(_compute_at_index)(i, w) for i in compute_idxs
            )
            # place back into the pre-allocated list in the same positions
            for i, feats in zip(compute_idxs, results):
                rows[i] = feats

        # forward-fill in between computed steps (same behavior)
        feats_df = pd.DataFrame(rows, index=idxs).ffill()
        feats_df = feats_df.add_suffix(f"_w{w}")  # same suffixing
        out = pd.concat([out, feats_df], axis=1)

    # numeric hygiene (unchanged)
    out = out.replace([np.inf, -np.inf], 0.0).fillna(0.0)

    # optional lag to prevent label leakage (unchanged)
    if shift_by_one:
        out = out.shift(1).fillna(0.0)

    return out

# ----------------- usage -----------------
# This assumes 'df_filtered' and 'df_volatility_features' are defined
df_vp = build_custom_vp_features(
    df_filtered,
    windows=[5, 10, 15, 30, 60],
    coverages=[0.40, 0.70],
    n_bins=24,
    topk=3,
    stride=5,            # speed-up knob
    shift_by_one=True,   # good for ML
    n_jobs=-1,           # <— parallel
    backend="threading"       # or "threading" to avoid pickling the df
)


assert len(df_vp) == len(df_filtered)
df_vp.isna().sum()



vp_unusual_inc_count_w5      0
vp_unusual_inc_maxz_w5       0
vp_unusual_inc_share_w5      0
vp_unusual_inc_near_px_w5    0
vp_unusual_dec_count_w5      0
                            ..
vp_vpoc_dist_atr_70_w60      0
vp_above_vah_70_w60          0
vp_below_val_70_w60          0
vp_entropy_w60               0
vp_topk_share_w60            0
Length: 170, dtype: int64

# 📊 Volume Profile Feature Summary

Each feature is computed per rolling window (`w ∈ {5,10,15,30,60}`)  
and receives a suffix `_{w}` — e.g., `_w5`, `_w10`, `_w15`, `_w30`, `_w60`.

---

## 🧭 Coverage-Specific Features (for each coverage level `{40, 70}`)
| Feature | Description |
|----------|--------------|
| **`vp_vpoc_{tag}`** | Volume Point of Control (VPOC) mid-price — bin with maximum aggregate volume. |
| **`vp_val_{tag}`** | Value Area Low (VAL) price boundary for coverage `{tag}`. |
| **`vp_vah_{tag}`** | Value Area High (VAH) price boundary for coverage `{tag}`. |
| **`vp_va_width_{tag}`** | Width of Value Area = `VAH − VAL` (in price units). |
| **`vp_va_width_pct_{tag}`** | Value-area width as a fraction of last close (`va_width / last_close`). |
| **`vp_vpoc_dist_{tag}`** | Distance from last close to VPOC (in price units). |
| **`vp_vpoc_dist_pct_{tag}`** | Relative VPOC distance (`vpoc_dist / last_close`). |
| **`vp_in_va_{tag}`** | 1 if last close ∈ [VAL, VAH], else 0. |
| **`vp_loc_{tag}`** | Location code of last close: −1 = below VAL, 0 = inside VA, +1 = above VAH. |
| **`vp_vpoc_dist_atr_{tag}`** | VPOC distance normalised by ATR (if available). |
| **`vp_above_vah_{tag}`** | 1 if last close > VAH, else 0. |
| **`vp_below_val_{tag}`** | 1 if last close < VAL, else 0. |

---

## 📈 Unusual Volume-Change Features
| Feature | Description |
|----------|--------------|
| **`vp_unusual_inc_count`** | Count of bins with unusually **increasing** volume (z ≥ 2). |
| **`vp_unusual_inc_maxz`** | Maximum z-score among increasing-volume bins. |
| **`vp_unusual_inc_share`** | Share of total volume contained in increasing-volume bins. |
| **`vp_unusual_inc_near_px`** | 1 if current price’s bin shows an increasing-volume anomaly. |
| **`vp_unusual_dec_count`** | Count of bins with unusually **decreasing** volume (z ≤ −2). |
| **`vp_unusual_dec_maxz`** | Maximum |z| among decreasing-volume bins. |
| **`vp_unusual_dec_share`** | Share of total volume in decreasing-volume bins. |
| **`vp_unusual_dec_near_px`** | 1 if current price’s bin shows a decreasing-volume anomaly. |

---

## 🧮 Profile Shape & Concentration
| Feature | Description |
|----------|--------------|
| **`vp_entropy`** | Entropy of the volume distribution (higher = more evenly spread). |
| **`vp_topk_share`** | Fraction of total volume held in the top K bins (K = 3). |

---

## 🧩 Example Naming by Window
| Example | Meaning |
|----------|----------|
| `vp_vpoc_70_w30` | 70 % coverage VPOC on 30-bar window. |
| `vp_in_va_40_w10` | Whether last close lies in 40 % value area on 10-bar window. |
| `vp_unusual_inc_share_w60` | Share of increasing-volume bins on 60-bar window. |

---

### ⚙️ Units & Conventions
- **Price features:** raw price units (e.g., index points).  
- **`*_pct` features:** relative to last close (e.g., 0.01 = 1 %).  
- **Flags:** binary 0/1 values.  
- **`vp_loc_*`:** −1 = below VAL, 0 = inside, +1 = above.  
- **ATR-normalised:** measured in “ATR units” (distance ÷ ATR).

---


### Seasonality & Time Encoding Features

In [13]:
# Seasonality & Time encoding (UTC)
# ===============================

def build_time_encodings_utc(
    df: pd.DataFrame,
    shift_by_one: bool = False   # usually not needed for time encodings
) -> pd.DataFrame:
    """
    Create cyclic time features in UTC:
      - tod_sin, tod_cos : time-of-day (0..24h) encoded on unit circle
      - dow_sin, dow_cos : day-of-week (Mon=0..Sun=6) encoded on unit circle

    Assumes df.index is a DateTimeIndex; if tz-naive, treats it as UTC.
    """

    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index must be a pandas.DatetimeIndex.")

    # Ensure UTC
    if df.index.tz is None:
        idx_utc = df.index.tz_localize("UTC")
    else:
        idx_utc = df.index.tz_convert("UTC")

    # ---- time of day (fraction of day) ----
    # use seconds for sub-minute timestamps as well
    seconds = (
        idx_utc.hour.astype(int) * 3600
        + idx_utc.minute.astype(int) * 60
        + idx_utc.second.astype(int)
    )
    # include microseconds if present
    if hasattr(idx_utc, "microsecond"):
        seconds = seconds + (idx_utc.microsecond.astype(int) / 1_000_000.0)

    tod_frac = seconds / 86_400.0                        # in [0,1)
    tod_ang  = 2.0 * np.pi * tod_frac
    tod_sin  = np.sin(tod_ang)
    tod_cos  = np.cos(tod_ang)

    # ---- day of week (Mon=0..Sun=6) ----
    dow = idx_utc.dayofweek.astype(float)               # 0..6
    dow_ang = 2.0 * np.pi * (dow / 7.0)
    dow_sin = np.sin(dow_ang)
    dow_cos = np.cos(dow_ang)

    out = pd.DataFrame(
        {
            "tod_sin": tod_sin,
            "tod_cos": tod_cos,
            "dow_sin": dow_sin,
            "dow_cos": dow_cos,
        },
        index=df.index,  # keep original index object
    ).astype(float)

    if shift_by_one:
        out = out.shift(1).fillna(0.0)

    return out

df_time = build_time_encodings_utc(df_filtered, shift_by_one=False)
assert len(df_time) == len(df_filtered)
print("df_time created with shape:", df_time.shape)
df_time.isna().sum()


df_time created with shape: (1691325, 4)


tod_sin    0
tod_cos    0
dow_sin    0
dow_cos    0
dtype: int64

### GARCH Volatility

In [14]:
# ===============================
# GARCH Volatility (best (p,q))
# ===============================
import numpy as np
import pandas as pd
import warnings
from joblib import Parallel, delayed
from arch import arch_model

# --------- helper: single (p,q) fit ----------
def _fit_garch_single(
    series: pd.Series,
    p: int,
    q: int,
    mean: str = "zero",      # "zero" is good for diffs/returns
    dist: str = "normal",    # "normal","t","skewt"
    rescale: bool = True,    # let arch rescale internally
    fit_options: dict | None = None
):
    """
    Fit one GARCH(p,q); return dict with metrics and conditional volatility.
    Returns None on failure.
    """
    try:
        am = arch_model(series, vol="GARCH", p=p, q=q, mean=mean, dist=dist, rescale=rescale)
        # safer fit defaults
        if fit_options is None:
            fit_options = {"disp": "off", "update_freq": 0}
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            res = am.fit(**fit_options)

        # Not all ARCH versions expose the same attrs — use only robust ones
        out = {
            "p": int(p),
            "q": int(q),
            "aic": float(getattr(res, "aic", np.inf)),
            "bic": float(getattr(res, "bic", np.inf)),
            "llf": float(getattr(res, "loglikelihood", np.nan)),
            "cond_vol": pd.Series(res.conditional_volatility, index=series.index).astype(float)
        }
        return out
    except Exception:
        return None


# --------- grid search + choose best ----------
def garch_grid_best(
    series: pd.Series,
    p_range=range(1, 6),
    q_range=range(1, 6),
    criterion: str = "bic",       # "bic", "aic" (min) or "llf" (max)
    mean: str = "zero",
    dist: str = "normal",
    rescale: bool = True,
    n_jobs: int = -1,
    backend: str = "loky",        # or "threading" on macOS if needed
    shift_by_one: bool = True,    # avoid leakage
    fill_value: float = 0.0,
    max_obs: int | None = 200_000,  # optionally limit obs for speed
    winsorize: bool = True,       # trim outliers for stability
    winsor_q: float = 0.001,      # 0.1% tails
    std_input: bool = True,       # standardize for numeric stability
    fit_options: dict | None = None
) -> tuple[pd.Series, dict]:
    """
    Fit GARCH(p,q) in parallel across p_range x q_range.
    Select best model by `criterion` and return (best_vol_series, best_meta).
    """
    s = pd.Series(series).astype(float).replace([np.inf, -np.inf], np.nan).dropna()

    # Optional subsample from the tail to speed up
    if max_obs is not None and len(s) > max_obs:
        s_fit = s.iloc[-max_obs:].copy()
    else:
        s_fit = s.copy()

    # Winsorize (trim extreme tails) to reduce optimizer issues
    if winsorize and len(s_fit) > 10:
        lo = s_fit.quantile(winsor_q)
        hi = s_fit.quantile(1 - winsor_q)
        s_fit = s_fit.clip(lower=lo, upper=hi)

    # Standardize (improves optimizer stability); keep scale to rescale back
    mean_s, std_s = (0.0, 1.0)
    if std_input and s_fit.std() > 0:
        mean_s = float(s_fit.mean())
        std_s = float(s_fit.std(ddof=0))
        s_fit = (s_fit - mean_s) / std_s

    # If variance is zero after cleaning, return flat series
    if s_fit.var(ddof=0) == 0 or not np.isfinite(s_fit.var(ddof=0)):
        best = pd.Series(fill_value, index=series.index)
        meta = {"p": 1, "q": 1, "criterion": criterion, "aic": np.inf, "bic": np.inf, "llf": np.nan}
        return (best.shift(1).fillna(fill_value) if shift_by_one else best, meta)

    # Parallel fits
    jobs = (
        delayed(_fit_garch_single)(s_fit, p, q, mean, dist, rescale, fit_options)
        for p in p_range for q in q_range
    )
    results = Parallel(n_jobs=n_jobs, backend=backend, verbose=0)(jobs)
    results = [r for r in results if r is not None and np.isfinite(r.get(criterion.lower(), np.inf) if criterion!="llf" else r.get("llf", -np.inf))]

    if not results:
        # Fallback: flat series
        best = pd.Series(fill_value, index=series.index)
        meta = {"p": None, "q": None, "criterion": criterion, "aic": np.inf, "bic": np.inf, "llf": np.nan}
        return (best.shift(1).fillna(fill_value) if shift_by_one else best, meta)

    # Choose best by criterion
    crit = criterion.lower()
    if crit in ("bic", "aic"):
        best_res = min(results, key=lambda r: r[crit])
    elif crit == "llf":
        best_res = max(results, key=lambda r: r["llf"])
    else:
        raise ValueError("criterion must be one of: 'bic','aic','llf'")

    # Take conditional volatility from the fit sample and align to full index
    vol_fit = best_res["cond_vol"]
    # If standardized input, rescale volatility to original units (vol scales by std)
    if std_input:
        vol_fit = vol_fit * std_s

    vol = vol_fit.reindex(series.index).ffill().fillna(fill_value)
    if shift_by_one:
        vol = vol.shift(1).fillna(fill_value)

    meta = {
        "p": best_res["p"],
        "q": best_res["q"],
        "criterion": criterion,
        "aic": best_res["aic"],
        "bic": best_res["bic"],
        "llf": best_res["llf"],
        "used_obs": int(len(s_fit))
    }
    return vol.astype(float), meta


# --------- convenience wrapper: build feature from prices ----------
def build_garch_vol_feature_from_prices(
    df: pd.DataFrame,
    price_col: str = "close",
    diff_kind: str = "diff",      # "diff" or "logret"
    **grid_kwargs
) -> tuple[pd.DataFrame, dict]:
    """
    Make stationary series from prices, run GARCH grid, return
    (DataFrame with 'garch_vol_best', meta).
    """
    prices = df[price_col].astype(float)
    if diff_kind == "diff":
        x = prices.diff()
    elif diff_kind == "logret":
        x = np.log(prices) - np.log(prices.shift(1))
    else:
        raise ValueError("diff_kind must be 'diff' or 'logret'")

    x = x.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    vol, meta = garch_grid_best(
        x,
        **grid_kwargs
    )
    df_feature = pd.DataFrame({"garch_vol_best": vol}, index=df.index)
    df_feature = df_feature.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    return df_feature, meta


# --------- usage with your df_filtered ----------
# Stationary input: use close price differences
# (You can switch backend to "threading" on macOS if loky pickling is slow)
df_garch_feature, meta = build_garch_vol_feature_from_prices(
    df_filtered,
    price_col="close",
    diff_kind="diff",
    p_range=range(1, 6),
    q_range=range(1, 6),
    criterion="bic",        # or "aic" / "llf"
    mean="zero",
    dist="normal",
    rescale=True,
    n_jobs=-1,
    backend="loky",
    shift_by_one=True,
    max_obs=200_000,        # speed knob
    winsorize=True,
    std_input=True,
    fit_options={"disp": "off", "update_freq": 0}  # you can add {"tol":1e-6,"options":{"maxiter":3000}}
)

print("Best GARCH params/meta:", meta)
assert len(df_garch_feature) == len(df_filtered)

df_garch_feature.isna().sum()

Best GARCH params/meta: {'p': 5, 'q': 5, 'criterion': 'bic', 'aic': 414513.2713824321, 'bic': 414625.53818153293, 'llf': -207245.63569121604, 'used_obs': 200000}


garch_vol_best    0
dtype: int64

### Candlestick Pattern Features

In [15]:
# --- FAST Candlestick Pattern Features (batched, low-fragmentation) ---
import numpy as np
import pandas as pd

def build_candlestick_features_fast(
    df: pd.DataFrame,
    windows: tuple[int, ...] = (5, 10, 15, 30, 60),
    shift_by_one: bool = True,
    normalize: bool = True,
    float_dtype: str = "float32",
) -> pd.DataFrame:
    need = {"open","high","low","close"}
    if not need.issubset(df.columns):
        raise ValueError(f"DataFrame must contain columns: {need}")

    O = df["open"].astype(float).to_numpy()
    H = df["high"].astype(float).to_numpy()
    L = df["low"].astype(float).to_numpy()
    C = df["close"].astype(float).to_numpy()
    idx = df.index

    try:
        import talib as ta
    except Exception as e:
        raise ImportError(
            "TA-Lib is required. Install with `pip install TA-Lib` "
            "(may need system libs on macOS/Linux)."
        ) from e

    patterns = [
        "CDLENGULFING","CDLHAMMER","CDLSHOOTINGSTAR","CDLDOJI",
        "CDLPIERCING","CDLDARKCLOUDCOVER","CDLMORNINGSTAR","CDLEVENINGSTAR",
        "CDL3WHITESOLDIERS","CDL3BLACKCROWS",
    ]

    # ---- raw & sign frames (built once, no per-column insert) ----
    raw_map = {}
    sign_map = {}
    for name in patterns:
        fn = getattr(ta, name)
        raw = fn(O, H, L, C).astype(np.int16)  # TA-Lib returns -100..100
        col_raw = f"{name.lower()}_raw"
        raw_map[col_raw] = raw

        if normalize:
            sign = (raw // 100).clip(-1, 1).astype(np.int8)
        else:
            sign = np.sign(raw).astype(np.int8)
        col_sign = f"{name.lower()}_sign"
        sign_map[col_sign] = sign

    raw_df  = pd.DataFrame(raw_map, index=idx)
    sign_df = pd.DataFrame(sign_map, index=idx)

    # Precompute boolean/int mats for rolling
    nz_df   = (sign_df != 0).astype(np.int8)
    bull_df = (sign_df == 1).astype(np.int8)
    bear_df = (sign_df == -1).astype(np.int8)

    # ---- rolling windows (batch ops), collect pieces, concat once ----
    pieces = [raw_df, sign_df]
    P = sign_df.shape[1]

    for w in windows:
        # presence / count
        any_df   = nz_df.rolling(w, min_periods=1).max().astype(np.int8)
        cnt_df   = nz_df.rolling(w, min_periods=1).sum().astype(np.int16)
        bullf_df = bull_df.rolling(w, min_periods=1).mean().astype(float_dtype)
        bearf_df = bear_df.rolling(w, min_periods=1).mean().astype(float_dtype)

        any_df.columns   = [f"{c}_any_w{w}"   for c in sign_df.columns]
        cnt_df.columns   = [f"{c}_count_w{w}" for c in sign_df.columns]
        bullf_df.columns = [f"{c}_bull_w{w}"  for c in sign_df.columns]
        bearf_df.columns = [f"{c}_bear_w{w}"  for c in sign_df.columns]
        pieces.extend([any_df, cnt_df, bullf_df, bearf_df])

        # aggregated strengths across patterns (single Series each)
        bull_strength = bull_df.rolling(w, min_periods=1).sum().sum(axis=1) / float(w * P)
        bear_strength = bear_df.rolling(w, min_periods=1).sum().sum(axis=1) / float(w * P)
        net_strength  = bull_strength - bear_strength

        pieces.append(bull_strength.astype(float_dtype).rename(f"bull_strength_w{w}").to_frame())
        pieces.append(bear_strength.astype(float_dtype).rename(f"bear_strength_w{w}").to_frame())
        pieces.append(net_strength.astype(float_dtype).rename(f"net_strength_w{w}").to_frame())

    out = pd.concat(pieces, axis=1, copy=False)
    out = out.replace([np.inf, -np.inf], 0.0)

    if shift_by_one:
        out = out.shift(1)

    out = out.fillna(0.0)
    # downcast floats to save RAM
    for c in out.columns:
        if pd.api.types.is_float_dtype(out[c]):
            out[c] = out[c].astype(float_dtype)

    # ensure no dup col names
    out = out.loc[:, ~out.columns.duplicated()].copy()
    return out

# ---------- usage ----------
df_cdl = build_candlestick_features_fast(
    df_filtered[["open","high","low","close"]],
    windows=(5, 10, 15, 30, 60),
    shift_by_one=True,
    normalize=True,
)
print("df_cdl created with shape:", df_cdl.shape)
assert len(df_cdl) == len(df_filtered)
df_cdl.isna().sum()


df_cdl created with shape: (1691325, 235)


cdlengulfing_raw                   0
cdlhammer_raw                      0
cdlshootingstar_raw                0
cdldoji_raw                        0
cdlpiercing_raw                    0
                                  ..
cdl3whitesoldiers_sign_bear_w60    0
cdl3blackcrows_sign_bear_w60       0
bull_strength_w60                  0
bear_strength_w60                  0
net_strength_w60                   0
Length: 235, dtype: int64

### Combine features to a DataFrame and write to file

In [16]:
# ---- pick the canonical index (your price DF) ----
base = df_filtered  # or whatever your master index DF is
idx = base.index

# ---- put every feature block in a list in one place ----
blocks = [
    df_norm_zscore,
    df_returns_variants,
    df_volatility_features,
    df_momentum_features,
    df_vol_z,
    df_vwap_features,
    df_microstructure,
    df_vp,
    df_time,
    df_garch_feature,
    df_cdl,
]

# optional: quick sanity checks before concat
for i, b in enumerate(blocks, 1):
    assert len(b) == len(idx), f"Block {i} length {len(b)} != base length {len(idx)}"
    # If you’re not 100% sure indices match exactly, force alignment:
    if not b.index.equals(idx):
        blocks[i-1] = b.reindex(idx)

# ---- single concat to avoid fragmentation ----
df_ultimate_feature_set = pd.concat(blocks, axis=1, copy=False)

# If you want to keep the index name/column from your base:
df_ultimate_feature_set = df_ultimate_feature_set.reindex(index=idx)

# OPTIONAL: if you’ve decided to zero-fill remaining NaNs
df_ultimate_feature_set = df_ultimate_feature_set.replace([np.inf, -np.inf], np.nan).fillna(0.0)


print("ultimate shape:", df_ultimate_feature_set.shape)


ultimate shape: (1691325, 727)


In [22]:
# Parquet (recommended)
df_ultimate_feature_set.to_parquet(
    "ultimate_feature_set.parquet",
    engine="fastparquet",          # or "fastparquet"
    compression="zstd",        # or "snappy"
    index=True
)

# If you still want CSV too
df_ultimate_feature_set.to_csv("ultimate_feature_set.csv", index=True)
